In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [14]:
movies = pd.read_csv('movies.csv')
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [15]:
columns_to_keep = ["id", "title", "overview", "genres", "keywords"]
movies = movies[columns_to_keep]

In [16]:
movies.head()

,id,title,overview,genres,keywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drug abuse exotic island east india trad...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy based on novel secret agent sequel mi6
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,based on novel mars medallion space travel pri...


In [17]:
movies.shape

(4803, 5)

In [18]:
movies = movies.dropna().sample(n=500).reset_index(drop=True)
movies.shape

(500, 5)

In [19]:
movies.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
dtype: int64

In [20]:
movies.to_csv('movies_sample.csv', index=False)

In [9]:
def clean_text(text):
    if isinstance(text, str):
        return " ".join(ast.literal_eval(text)) if text.startswith("[") else text
    return ""

In [ ]:
movies["genres"] = movies["genres"].apply(clean_text)
movies["keywords"] = movies["keywords"].apply(clean_text)
movies.head()

,id,title,overview,genres,keywords
0,50601,5 Days of War,An American journalist and his cameraman are c...,War Drama,journalist interpreter georgia europe war zone
1,10705,Henry V,Gritty adaption of William Shakespeare's play ...,War Drama History Action Romance,shakespeare hero kingdom war based on play
2,23988,Stan Helsing,It's Halloween night and video store clerk Sta...,Comedy Horror,halloween slapstick duringcreditsstinger
3,13811,Knowing,A teacher opens a time capsule that has been d...,Action Adventure Drama Mystery Science Fiction,cataclysm code suspense end of the world time ...
4,32856,Valentine's Day,More than a dozen Angelenos navigate Valentine...,Comedy Romance,flower married couple florist kiss single


In [ ]:
movies["combined_text"] = movies["genres"] + " " + \
    movies["keywords"] + " " + movies["overview"]
movies.head()

,id,title,overview,genres,keywords,combined_text
0,50601,5 Days of War,An American journalist and his cameraman are c...,War Drama,journalist interpreter georgia europe war zone,War Drama journalist interpreter georgia europ...
1,10705,Henry V,Gritty adaption of William Shakespeare's play ...,War Drama History Action Romance,shakespeare hero kingdom war based on play,War Drama History Action Romance shakespeare h...
2,23988,Stan Helsing,It's Halloween night and video store clerk Sta...,Comedy Horror,halloween slapstick duringcreditsstinger,Comedy Horror halloween slapstick duringcredit...
3,13811,Knowing,A teacher opens a time capsule that has been d...,Action Adventure Drama Mystery Science Fiction,cataclysm code suspense end of the world time ...,Action Adventure Drama Mystery Science Fiction...
4,32856,Valentine's Day,More than a dozen Angelenos navigate Valentine...,Comedy Romance,flower married couple florist kiss single,Comedy Romance flower married couple florist k...


In [30]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words="english")

# Fit and transform the movie descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(movies["combined_text"])

In [ ]:
import pandas as pd
import ast
import argparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def clean_text(text):
    """Cleans text by converting JSON-like strings into readable text."""
    if isinstance(text, str):
        return " ".join(ast.literal_eval(text)) if text.startswith("[") else text
    return ""


def preprocess_data(filepath):
    """Loads and preprocesses the dataset."""
    movies = pd.read_csv(filepath)

    # Clean genres and keywords
    movies["genres"] = movies["genres"].apply(clean_text)
    movies["keywords"] = movies["keywords"].apply(clean_text)

    # Combine text fields
    movies["combined_text"] = movies["genres"] + " " + \
        movies["keywords"] + " " + movies["overview"]

    # Drop rows with missing descriptions
    movies = movies.dropna(subset=["combined_text"])

    return movies


def train_tfidf(movies):
    """Fits a TF-IDF vectorizer and returns the matrix."""
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(movies["combined_text"])
    return vectorizer, tfidf_matrix


def recommend_movies(query, movies, vectorizer, tfidf_matrix, top_n=5):
    """Recommends top N similar movies based on a text query."""
    query_vec = vectorizer.transform([query])
    similarity_scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    recommendations = movies.iloc[top_indices][["title", "overview"]]

    # Format output neatly
    formatted_output = "\nTop Recommendations:\n"
    for index, row in recommendations.iterrows():
        formatted_output += f"\nTitle: {row['title']}\nDescription: {row['overview']}\n{'-'*80}"
    return formatted_output


def main():
    parser = argparse.ArgumentParser(
        description="Content-Based Movie Recommendation System")
    parser.add_argument("query", type=str,
                        help="User's movie description for recommendations")
    args = parser.parse_args()

    # Load and preprocess data
    movies = preprocess_data("movies_cleaned.csv")
    vectorizer, tfidf_matrix = train_tfidf(movies)

    # Get recommendations
    recommendations = recommend_movies(
        args.query, movies, vectorizer, tfidf_matrix)

    # Print results
    print(recommendations)


if __name__ == "__main__":
    main()

In [54]:
user_query = "I like to watch romantic movies"
recommendations = recommend_movies(user_query)

print("Top Recommendations:")
print(recommendations['overview'].values)

Top Recommendations:
['The mostly true story of the legendary "worst director of all time", who, with the help of his strange friends, filmed countless B-movies without ever becoming famous or successful.'
 'This documentary follows two inner-city Chicago residents, Arthur Agee and William Gates, as they follow their dreams of becoming basketball superstars. Beginning at the start of their high school years, and ending almost 5 years later, as they start college, we watch the boys mature into men, still retaining their "Hoop Dreams".'
 'Bored with Bollywood movies but fascinated with their Hollywood counterparts from his youth, Ram dreams to become a singer and actor in America, the country where dreams are made. He is encouraged when his American-based close friend, Vijay Rao, comes for visit, and brags about driving a Mercedes and living in a penthouse.'
 'A small town waitress gets a nail accidentally lodged in her head causing unpredictable behavior that leads her to Washington, DC

In [2]:
movies.to_csv('movies_sample.csv', index=False)

NameError: name 'movies' is not defined